In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

def get_session(gpu_fraction=0.3):
    '''Assume that you have 6GB of GPU memory and want to allocate ~2GB'''
    num_threads = os.environ.get('OMP_NUM_THREADS')
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)

    if num_threads:
        return tf.Session(config=tf.ConfigProto(
            gpu_options=gpu_options, intra_op_parallelism_threads=num_threads))
    else:
        return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

KTF.set_session(get_session(0.5))

Using TensorFlow backend.


In [2]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np

import os

In [3]:
batch_size = 16
num_classes = 3
epochs = 5000

# num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_neuron_classification_trained_model.h5'

In [4]:
# write a function to select x_train y_train x_test y_test
data_x = np.load('image_data_256x256.npy')
data_y = np.load('labels.npy')
data_x = np.swapaxes(np.swapaxes(data_x,1,2),2,3)
data_y = np.array([data_y]).T

from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2)
print x_train.shape, x_test.shape, y_train.shape, y_test.shape
# shuffle the data
# select 20% as the test data and 80% as the training data
# np.random.choice(len(data_x), len(data_x) * 0.2, replace=False)

/home/edward/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(221, 256, 256, 2) (56, 256, 256, 2) (221, 1) (56, 1)


In [5]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print x_train.shape, x_test.shape, y_train.shape, y_test.shape

(221, 256, 256, 2) (56, 256, 256, 2) (221, 3) (56, 3)


In [6]:
# define the model 
# CNN structure
model = Sequential()
model.add(Conv2D(16, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [7]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 16)      304       
_________________________________________________________________
activation_1 (Activation)    (None, 256, 256, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 254, 254, 16)      2320      
_________________________________________________________________
activation_2 (Activation)    (None, 254, 254, 16)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 16)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 127, 127, 16)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 127, 127, 64)      9280      
__________

In [8]:
data_augmentation = True

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)
      
    # Fit the model on the batches generated by datagen.flow().
    for _ in range(1):
        model.fit_generator(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train.shape[0] // batch_size,
                            epochs=epochs,
                            validation_data=(x_test, y_test),
                            workers=4,
                            callbacks=None)
#         model.evaluate(x=x_test, y=y_test, batch_size=batch_size)

Using real-time data augmentation.


/home/edward/anaconda2/lib/python2.7/site-packages/keras/preprocessing/image.py:653: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (221, 256, 256, 2) (2 channels).
  ' (' + str(x.shape[self.channel_axis]) + ' channels).')
/home/edward/anaconda2/lib/python2.7/site-packages/keras/preprocessing/image.py:787: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (221, 256, 256, 2) (2 channels).
  ' (' + str(self.x.shape[channels_axis]) + ' channels).')


Epoch 1/5000
13/13 [==============================] - 6s - loss: 0.9969 - acc: 0.5579 - val_loss: 0.9662 - val_acc: 0.6429
Epoch 2/5000
13/13 [==============================] - 3s - loss: 0.9923 - acc: 0.5817 - val_loss: 0.9662 - val_acc: 0.6429
Epoch 3/5000
13/13 [==============================] - 3s - loss: 0.9876 - acc: 0.5626 - val_loss: 0.9709 - val_acc: 0.6429
Epoch 4/5000
13/13 [==============================] - 3s - loss: 0.9751 - acc: 0.5962 - val_loss: 0.9452 - val_acc: 0.6429
Epoch 5/5000
13/13 [==============================] - 3s - loss: 0.9486 - acc: 0.6002 - val_loss: 0.9423 - val_acc: 0.6429
Epoch 6/5000
13/13 [==============================] - 3s - loss: 1.0201 - acc: 0.5442 - val_loss: 0.9633 - val_acc: 0.6429
Epoch 7/5000
13/13 [==============================] - 3s - loss: 0.9337 - acc: 0.6050 - val_loss: 0.9339 - val_acc: 0.6429
Epoch 8/5000
13/13 [==============================] - 3s - loss: 0.9779 - acc: 0.5607 - val_loss: 0.9379 - val_acc: 0.6429
Epoch 9/5000
13/

13/13 [==============================] - 3s - loss: 0.6831 - acc: 0.7350 - val_loss: 0.6191 - val_acc: 0.7500
Epoch 68/5000
13/13 [==============================] - 3s - loss: 0.7434 - acc: 0.6958 - val_loss: 0.7444 - val_acc: 0.6607
Epoch 69/5000
13/13 [==============================] - 3s - loss: 0.7602 - acc: 0.7109 - val_loss: 0.7759 - val_acc: 0.7143
Epoch 70/5000
13/13 [==============================] - 3s - loss: 0.6798 - acc: 0.7167 - val_loss: 0.6018 - val_acc: 0.7321
Epoch 71/5000
13/13 [==============================] - 3s - loss: 0.7145 - acc: 0.7212 - val_loss: 0.6956 - val_acc: 0.7321
Epoch 72/5000
13/13 [==============================] - 3s - loss: 0.6984 - acc: 0.6941 - val_loss: 0.6910 - val_acc: 0.7321
Epoch 73/5000
13/13 [==============================] - 3s - loss: 0.7214 - acc: 0.7260 - val_loss: 0.6547 - val_acc: 0.7500
Epoch 74/5000
13/13 [==============================] - 3s - loss: 0.7730 - acc: 0.6820 - val_loss: 0.6397 - val_acc: 0.7500
Epoch 75/5000
13/13 [=

13/13 [==============================] - 3s - loss: 0.5694 - acc: 0.7649 - val_loss: 0.9385 - val_acc: 0.5893
Epoch 133/5000
13/13 [==============================] - 3s - loss: 0.5124 - acc: 0.8034 - val_loss: 1.1647 - val_acc: 0.4643
Epoch 134/5000
13/13 [==============================] - 3s - loss: 0.6887 - acc: 0.7401 - val_loss: 0.6651 - val_acc: 0.7321
Epoch 135/5000
13/13 [==============================] - 3s - loss: 0.5940 - acc: 0.7542 - val_loss: 1.1200 - val_acc: 0.4464
Epoch 136/5000
13/13 [==============================] - 3s - loss: 0.6022 - acc: 0.7707 - val_loss: 0.5242 - val_acc: 0.7500
Epoch 137/5000
13/13 [==============================] - 3s - loss: 0.5980 - acc: 0.7707 - val_loss: 0.8581 - val_acc: 0.6250
Epoch 138/5000
13/13 [==============================] - 3s - loss: 0.6268 - acc: 0.7525 - val_loss: 0.5690 - val_acc: 0.6964
Epoch 139/5000
13/13 [==============================] - 3s - loss: 0.5425 - acc: 0.7659 - val_loss: 0.9743 - val_acc: 0.5536
Epoch 140/5000


...

13/13 [==============================] - 3s - loss: 0.0655 - acc: 0.9711 - val_loss: 0.7674 - val_acc: 0.8571
Epoch 4683/5000
13/13 [==============================] - 3s - loss: 0.0050 - acc: 1.0000 - val_loss: 0.7542 - val_acc: 0.8750
Epoch 4684/5000
13/13 [==============================] - 3s - loss: 0.0626 - acc: 0.9797 - val_loss: 0.7690 - val_acc: 0.8571
Epoch 4685/5000
13/13 [==============================] - 3s - loss: 0.0955 - acc: 0.9759 - val_loss: 0.3715 - val_acc: 0.9464
Epoch 4686/5000
13/13 [==============================] - 3s - loss: 0.0156 - acc: 0.9904 - val_loss: 0.8042 - val_acc: 0.8393
Epoch 4687/5000
13/13 [==============================] - 3s - loss: 0.0585 - acc: 0.9797 - val_loss: 0.4647 - val_acc: 0.8571
Epoch 4688/5000
13/13 [==============================] - 3s - loss: 0.0644 - acc: 0.9856 - val_loss: 0.2707 - val_acc: 0.8929
Epoch 4689/5000
13/13 [==============================] - 3s - loss: 0.1397 - acc: 0.9471 - val_loss: 0.5738 - val_acc: 0.8750
Epoch 46

13/13 [==============================] - 3s - loss: 0.0208 - acc: 0.9952 - val_loss: 0.6904 - val_acc: 0.8750
Epoch 4748/5000
13/13 [==============================] - 3s - loss: 0.2131 - acc: 0.9759 - val_loss: 0.9190 - val_acc: 0.8214
Epoch 4749/5000
13/13 [==============================] - 3s - loss: 0.0708 - acc: 0.9760 - val_loss: 0.4094 - val_acc: 0.9107
Epoch 4750/5000
13/13 [==============================] - 3s - loss: 0.0034 - acc: 1.0000 - val_loss: 0.5840 - val_acc: 0.8571
Epoch 4751/5000
13/13 [==============================] - 3s - loss: 0.0545 - acc: 0.9807 - val_loss: 0.9039 - val_acc: 0.8036
Epoch 4752/5000
13/13 [==============================] - 3s - loss: 0.1151 - acc: 0.9759 - val_loss: 0.4447 - val_acc: 0.9107
Epoch 4753/5000
13/13 [==============================] - 3s - loss: 0.0055 - acc: 1.0000 - val_loss: 0.8870 - val_acc: 0.8214
Epoch 4754/5000
13/13 [==============================] - 3s - loss: 0.0085 - acc: 0.9952 - val_loss: 0.9472 - val_acc: 0.8036
Epoch 47

13/13 [==============================] - 3s - loss: 0.0651 - acc: 0.9663 - val_loss: 0.3483 - val_acc: 0.9286
Epoch 4813/5000
13/13 [==============================] - 3s - loss: 0.0112 - acc: 0.9952 - val_loss: 0.5615 - val_acc: 0.8750
Epoch 4814/5000
13/13 [==============================] - 3s - loss: 0.0663 - acc: 0.9797 - val_loss: 0.8373 - val_acc: 0.8214
Epoch 4815/5000
13/13 [==============================] - 3s - loss: 0.0341 - acc: 0.9904 - val_loss: 1.7217 - val_acc: 0.7679
Epoch 4816/5000
13/13 [==============================] - 3s - loss: 0.0463 - acc: 0.9856 - val_loss: 1.2963 - val_acc: 0.8214
Epoch 4817/5000
13/13 [==============================] - 3s - loss: 0.1001 - acc: 0.9711 - val_loss: 0.9982 - val_acc: 0.8214
Epoch 4818/5000
13/13 [==============================] - 3s - loss: 0.0108 - acc: 1.0000 - val_loss: 0.6256 - val_acc: 0.8214
Epoch 4819/5000
13/13 [==============================] - 3s - loss: 0.0015 - acc: 1.0000 - val_loss: 0.8831 - val_acc: 0.8214
Epoch 48

13/13 [==============================] - 3s - loss: 0.0637 - acc: 0.9807 - val_loss: 0.4777 - val_acc: 0.8571
Epoch 4878/5000
13/13 [==============================] - 3s - loss: 0.0860 - acc: 0.9759 - val_loss: 0.7938 - val_acc: 0.8393
Epoch 4879/5000
13/13 [==============================] - 3s - loss: 0.0660 - acc: 0.9845 - val_loss: 0.8711 - val_acc: 0.8393
Epoch 4880/5000
13/13 [==============================] - 3s - loss: 0.0029 - acc: 1.0000 - val_loss: 0.5997 - val_acc: 0.8929
Epoch 4881/5000
13/13 [==============================] - 3s - loss: 0.1761 - acc: 0.9663 - val_loss: 0.7127 - val_acc: 0.8571
Epoch 4882/5000
13/13 [==============================] - 3s - loss: 0.0366 - acc: 0.9845 - val_loss: 0.7169 - val_acc: 0.8750
Epoch 4883/5000
13/13 [==============================] - 3s - loss: 0.0120 - acc: 0.9952 - val_loss: 0.5815 - val_acc: 0.8750
Epoch 4884/5000
13/13 [==============================] - 3s - loss: 0.0062 - acc: 1.0000 - val_loss: 0.4738 - val_acc: 0.9286
Epoch 48

13/13 [==============================] - 3s - loss: 0.0029 - acc: 1.0000 - val_loss: 0.8770 - val_acc: 0.8036
Epoch 4943/5000
13/13 [==============================] - 3s - loss: 0.0024 - acc: 1.0000 - val_loss: 0.4794 - val_acc: 0.9107
Epoch 4944/5000
13/13 [==============================] - 3s - loss: 0.1412 - acc: 0.9712 - val_loss: 1.5423 - val_acc: 0.8214
Epoch 4945/5000
13/13 [==============================] - 3s - loss: 0.0182 - acc: 0.9904 - val_loss: 1.6857 - val_acc: 0.7500
Epoch 4946/5000
13/13 [==============================] - 3s - loss: 0.1029 - acc: 0.9739 - val_loss: 0.6790 - val_acc: 0.8393
Epoch 4947/5000
13/13 [==============================] - 3s - loss: 0.0122 - acc: 0.9952 - val_loss: 1.3069 - val_acc: 0.8036
Epoch 4948/5000
13/13 [==============================] - 3s - loss: 0.0324 - acc: 0.9904 - val_loss: 1.2752 - val_acc: 0.8036
Epoch 4949/5000
13/13 [==============================] - 3s - loss: 0.0413 - acc: 0.9856 - val_loss: 1.6231 - val_acc: 0.7500
Epoch 49

In [ ]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])